In [53]:
SCHOOL = "USF"
model = "nltk_sia" # models can be "nltk_sia" or "bart"
data_path = f"../bias_processing/data/1/{SCHOOL.lower()}_dataset.country.csv"
output_path = f"../bias_processing/data/2/{model}/{SCHOOL.lower()}_dataset_granularity.country.csv"

In [54]:
"""
Load in a csv from Sentiment_Dataset_Maker and add 4x3x3 columns
4 topics ("Israel", "Palestine", "India", "China")
3 hypotheses for sentiment (Positive, Negative, Neutral)
3 levels of granularity
Compute sentiment for entire article
Compute sentiment for each paragraph in an article, take maximum for Positive and Negative, average for neutral
Make 'average' and 'max' options to a function call so we can change if need be
Compute sentiment for each sentence in an article, similar to paragraph approach
Save a new csv with these added columns
"""

'\nLoad in a csv from Sentiment_Dataset_Maker and add 4x3x3 columns\n4 topics ("Israel", "Palestine", "India", "China")\n3 hypotheses for sentiment (Positive, Negative, Neutral)\n3 levels of granularity\nCompute sentiment for entire article\nCompute sentiment for each paragraph in an article, take maximum for Positive and Negative, average for neutral\nMake \'average\' and \'max\' options to a function call so we can change if need be\nCompute sentiment for each sentence in an article, similar to paragraph approach\nSave a new csv with these added columns\n'

In [55]:
%pip install transformers nltk

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [56]:
import nltk
nltk.download('vader_lexicon')
nltk.download('punkt')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/Dana/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to /Users/Dana/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [57]:
import pandas as pd
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from transformers import pipeline
import nltk

# Function to return the sentiment of a text
def get_sentiment(text, granularity, keyword, model=model, method='avg'):
    if model == "nltk_sia":
        # Instantiate the sentiment analyzer
        sia = SentimentIntensityAnalyzer()
        # Output is a dict containing {'neg','pos','neu','composition'}. First three are needed for all future functionality
        def get_model_scores(text):
            scores = sia.polarity_scores(text)
            return scores
        def get_keys(text):
            return sia.polarity_scores(text).keys()

    elif model == "bart":
        bart_analyzer = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
        def get_model_scores(text):
            labels = ['positive', 'negative', 'neutral']
            scores = bart_analyzer(text, labels)['scores']
            scores = {'pos': scores[0], 'neg': scores[1], 'neu': scores[2]}
            return scores
        def get_keys(text):
            return ['pos', 'neg', 'neu']

    if granularity == 'article':
        scores = get_model_scores(text)
        return scores['neg'], scores['pos'], scores['neu']
    elif granularity in ['paragraph', 'sentence']:
        if granularity == 'paragraph':
            # Calculate the polarity scores for each paragraph and store them in a list
            # TODO: Revise and check paragraph splitting, may have issues with article splitting
            listed_scores = [get_model_scores(paragraph) for paragraph in text.split('\n') if paragraph]
        elif granularity == 'sentence':
            listed_scores = [get_model_scores(sentence) for sentence in nltk.sent_tokenize(text)]

        # Transpose the list of dictionaries to separate the values for each key
        transposed_scores = list(zip(*[d.values() for d in listed_scores]))

        # Find the maximum value for each key using the max function
        ranked_scores = [max(scores) for scores in transposed_scores]

        # Create a dictionary with the corresponding keys and maximum values
        result_dict = dict(zip(get_keys(text), ranked_scores))

        return result_dict['neg'], result_dict['pos'], result_dict['neu']

# # Load the csv
# df = pd.read_csv(data_path)

# # Process sentiment analysis
# for granularity in ['article', 'paragraph', 'sentence']:
#     print(granularity)
#     df[f'{granularity}_neg'], df[f'{granularity}_pos'], df[f'{granularity}_neu'] = zip(
#         *df.apply(lambda row: get_sentiment(row['article'], granularity, row['keyword'], model=model), axis=1)
#     )

# # Save the output DataFrame into a new CSV file
# df.to_csv(output_path, index=False)


In [58]:
from sentiment_calculater import build_csv

build_csv(data_path, output_path, get_sentiment)

Article: As coach Jim Leavitt stood on ... | Keyword: China | Granularity: article | Neg: 0.078 | Pos: 0.17 | Neu: 0.752
Article: As coach Jim Leavitt stood on ... | Keyword: China | Granularity: paragraph | Neg: 0.23 | Pos: 0.359 | Neu: 1.0
Article: As coach Jim Leavitt stood on ... | Keyword: China | Granularity: sentence | Neg: 0.402 | Pos: 0.355 | Neu: 1.0
Article: YouTube is no longer just for ... | Keyword: China | Granularity: article | Neg: 0.017 | Pos: 0.135 | Neu: 0.848
Article: YouTube is no longer just for ... | Keyword: China | Granularity: paragraph | Neg: 0.106 | Pos: 0.337 | Neu: 0.94
Article: YouTube is no longer just for ... | Keyword: China | Granularity: sentence | Neg: 0.285 | Pos: 0.422 | Neu: 1.0
Article: America has been a great natio... | Keyword: China | Granularity: article | Neg: 0.117 | Pos: 0.13 | Neu: 0.753
Article: America has been a great natio... | Keyword: China | Granularity: paragraph | Neg: 0.227 | Pos: 0.215 | Neu: 1.0
Article: America has been a 

KeyError: 'neg'